In [8]:
# Imports
import whisper
import os
from pydub import AudioSegment
import ntpath
import shutil
from scipy.io import wavfile
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import datetime
import datetime
import shutil
import ffmpeg

In [9]:
# Set Paths
recording_path = ""
ct = datetime.datetime.now()
safe_path = path = os.path.expanduser(os.path.join("~", "radio_analyzer", str(ct).replace(":", "-").replace(" ", "-").replace(".", "-")))
os.makedirs(safe_path)

In [10]:
#Function to create a Audiochunk with start and endtime
def splitter(recording: AudioSegment, section_start, section_finish, section_counter, folder_path, format):
    working_part = recording[section_start:section_finish]
    working_part.export(os.path.join(folder_path, "{:06d}".format(section_counter) + ".mp3"), format=format)

In [11]:
# Splitt Audiofile in chunks

# Get format, load file, create Outputfolder
file_format = recording_path.split('.', 1)[1]
print(recording_path)
recording = AudioSegment.from_file(recording_path, format=recording_path.split('.', 1)[1])

# Create Folder for Output
name_arg_2 = ntpath.basename(recording_path).split(".", 1)[0]
tmp = recording_path.split("\\")
name_arg_1 = tmp[len(tmp) - 3] + "-" + tmp[len(tmp) - 2]
folder_path = os.path.join(safe_path, name_arg_1 + "-" + name_arg_2)

# Remove preexisting old files
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
os.makedirs(folder_path)

full_length = len(recording)
section_start, section_finish, section_counter = 0, 30000, 1
if full_length < section_finish:
    print("File is to short and needs no splitting")
    recording.export(os.path.join(folder_path, "{:06d}".format(section_counter) + ".mp3"), format=file_format)
else:
    while section_finish <= full_length:
        splitter(recording, section_start, section_finish, section_counter, folder_path, file_format)
        section_start = section_finish - 2000
        section_finish = section_finish + 28000
        section_counter += 1
    section_finish = full_length
    splitter(recording, section_start, section_finish, section_counter, folder_path, file_format)

D:\Dropbox\Studium\Masterarbeit\Recordings mit Quellen\prigoshin\mp3\001.mp3


In [12]:
# Transcribe and Translate
"""
Sollen die txt files für wer benutzt werden, internal_mode bitte auf true setzen.
Das entfernt die Abgrenzungs- und erklärungsstrings.
"""
path = folder_path
internal_mode = False

# Helsinki-nlp
tokenizer_helsinki = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model_helsinki = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")

# Whisper

whisper_model = "large"

filename = path.rsplit("\\", 1)[1]
print("File: " + filename)

text_original = []
text_english = []
text_german = []

if not internal_mode:
    text_original.append(("Model: whisper-" + whisper_model + " Task: transcribe\n").encode('utf-8'))
    text_original.append(("###START OF ORIGINAL TRANSCRIPTION FROM FILE " + str(filename) + "###").encode('utf-8'))
    text_english.append("Model: whisper-large-v2 Task: translate original-english\n")
    text_english.append("###START OF ENGLISH TRANSLATION FROM FILE " + str(filename) + "###")
    text_german.append("Model: Helsinki-nlp Task: translate english-german\n")
    text_german.append("###START OF GERMAN TRANSLATION FROM FILE " + str(filename) + "###")

model = whisper.load_model(whisper_model)

time_start = 0
time_end = 30

for file in os.listdir(path):
    print("Working on part: " + file)

    if not internal_mode:
        text_original.append(("\n######## START OF " + str(os.path.basename(file)).upper()
                            + " (" + str(datetime.timedelta(seconds=time_start)) + " - "
                            + str(datetime.timedelta(seconds=time_end)) + "s) " + "########\n").encode('utf-8'))
        text_english.append("\n######## START OF " + str(os.path.basename(file)).upper()
                            + " (" + str(datetime.timedelta(seconds=time_start))
                            + " - " + str(datetime.timedelta(seconds=time_end)) + "s) " + "########\n")
        text_german.append("\n######## START OF " + str(os.path.basename(file)).upper()
                            + " (" + str(datetime.timedelta(seconds=time_start)) + " - "
                            + str(datetime.timedelta(seconds=time_end)) + "s) " + "########\n")
        

    transcript = model.transcribe(os.path.join(path, file), task="transcribe")["text"].encode('utf-8')
    translate = model.transcribe(os.path.join(path, file), task="translate")["text"]

    text_original.append(transcript)

    text_english.append(translate)

    # Helsinki

    input_ids_helsinki = tokenizer_helsinki.encode(text_english, return_tensors="pt")
    outputs_helsinki = model_helsinki.generate(input_ids_helsinki)
    decoded_helsinki = tokenizer_helsinki.decode(outputs_helsinki[0], skip_special_tokens=True)
        
    text_german.append(decoded_helsinki)

    time_start = time_end - 2
    time_end = time_start + 30

eng_out = [x.encode('utf-8') for x in text_english]
deu_out = [x.encode('utf-8') for x in text_german]

with open(os.path.join(path, "translation_english.txt"), 'wb') as f:
    for entry in eng_out:
        f.write(entry)
f.close()
with open(os.path.join(path, "transcript_original.txt"), 'wb') as f:
    for entry in text_original:
        f.write(entry)
f.close()
with open(os.path.join(path, "translation_german.txt"), 'wb') as f:
    for entry in deu_out:
        f.write(entry)
f.close()
print("And we are done :) \nYou can find your files here: " + folder_path )


File: prigoshin-mp3-001
Working on part: 000001.mp3
Working on part: 000002.mp3
Working on part: 000003.mp3
And we are done here :) 
 You can find your files here: C:\Users\alatt\radio_analyzer\2023-07-06-23-21-29-027576\prigoshin-mp3-001
